In [11]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

In [12]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        #if i==0:
        #  arches[k]="s->"+str(j+1)
        #elif j==0:
        #  arches[k]=str(i+1)+"->s"
        #elif i==columns-1:
        #  arches[k]="t->"+str(j+1)
        #elif j==columns-1:
        #  arches[k]=str(i+1)+"->t"
        #else:
        #  arches[k]=str(i+1)+"->"+str(j+1)
        arches[k]=str(i+1)+"->"+str(j+1)
        k+=1
  return NA, arches


def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]


def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

In [13]:

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])


C = np.array([2, 2, 2, 5, 2, 1, 2])

Aeq, arc_idxs = NN2NA(NN)
beq = np.array([1, 0, 0, 0, 0, -1])

max_q=None
bounds = tuple([(0, max_q) for arcs in range(0, Aeq.shape[1])])
t = [np.array([3, 1, 3, 1, 3, 3, 5])]
#t = np.transpose(t)

T=9


print('## Optimizer inputs ## \n'
      'Cost vector: %s \n\n '
      'Arches in order:\n%s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

## Optimizer inputs ## 
Cost vector: [2 2 2 5 2 1 2] 

 Arches in order:
['1->2', '1->3', '2->4', '2->6', '3->5', '4->6', '5->6'] 

A_eq Node-Arc matrix:
[[1.00 1.00 0.00 0.00 0.00 0.00 0.00]
 [-1.00 0.00 1.00 1.00 0.00 0.00 0.00]
 [0.00 -1.00 0.00 0.00 1.00 0.00 0.00]
 [0.00 0.00 -1.00 0.00 0.00 1.00 0.00]
 [0.00 0.00 0.00 0.00 -1.00 0.00 1.00]
 [0.00 0.00 0.00 -1.00 0.00 -1.00 -1.00]] 

b_eq demand-supply vector: [ 1  0  0  0  0 -1] 

Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None))


In [14]:
# OPTIMIZE:

res = linprog(C, A_eq=Aeq, b_eq=beq, A_ub=t, b_ub=T, bounds=bounds, method='interior-point')  #simplex get negative solution near 0. strange.

# GET THE SOLUTION:
#usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1



np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})


selarcs = get_selected_arcs(arc_idxs, res.x)

total_time=0
for i in selarcs:
  index=arc_idxs.index(i)
  total_time= total_time+t[0][index]

print('The raw solution will be: %s' % res.x)
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f' % res.fun)
print('The time would be:',total_time,'\n\n')

The raw solution will be: [1.00 0.00 1.00 0.00 0.00 1.00 0.00]
The arcs that make the shortest path will be (from, to): ['1->2', '2->4', '4->6']
The minimum cost will be: 5.00
The time would be: 9 




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [15]:
T=8

res = linprog(C, A_eq=Aeq, b_eq=beq, A_ub=t, b_ub=T, bounds=bounds, method='interior-point')  #simplex get negative solution near 0. strange.

# GET THE SOLUTION:
#usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1



np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})


selarcs = get_selected_arcs(arc_idxs, res.x)

total_time=0
for i in selarcs:
  index=arc_idxs.index(i)
  total_time= total_time+t[0][index]

print('The arches are:          :',arc_idxs)
print('The raw solution will be:    %s' % res.x)
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f' % res.fun)
print('The time would be:',total_time,'\n\n')
print('The algorithm does not resolve ok the problem.')


The arches are:          : ['1->2', '1->3', '2->4', '2->6', '3->5', '4->6', '5->6']
The raw solution will be:    [1.00 0.00 0.80 0.20 0.00 0.80 0.00]
The arcs that make the shortest path will be (from, to): ['1->2']
The minimum cost will be: 5.40
The time would be: 3 


The algorithm does not resolve ok the problem.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [16]:
print('As the result is not as expected, we could implement Integer Linear Programming.

We can apply an other condition that would be reduce the quantity of arches activated. Bu we should need a new restriction. 
The arches are:          : ['1->2', '1->3', '2->4', '2->6', '3->5', '4->6', '5->6']
The raw solution will be:    [1.00 0.00 0.80 0.20 0.00 0.80 0.00]
The arcs that make the shortest path will be (from, to): ['1->2']
The minimum cost will be: 5.40
The time would be: 3 


The algorithm does not resolve ok the problem.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
